In [31]:
import pandas as pd
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split

In [2]:
import lightgbm as lgb

# LightGBM model and parameters

In [72]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 31
params['min_data'] = 50
params['max_depth'] = -1
params['device'] = 'cpu' 

In [4]:
# Prepare datasets
X = pd.read_csv('data/app_train_processed.csv')
test_data = pd.read_csv('data/app_test_processed.csv')
Y  = X['TARGET']
X  = X.drop(columns=['TARGET'],axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [58]:
d_train = lgb.Dataset(X_train, label=y_train)

In [73]:
clf = lgb.train(params, d_train, 200)

In [74]:
y_pred=clf.predict(X_test)

In [75]:
type(y_test.values)
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [76]:
auc(fpr, tpr)

0.7423986911430639

# Predict test data


In [33]:
predict_data = clf.predict(test_data)

In [44]:
df_pred = pd.DataFrame(predict_data)
df_pred['SK_ID_CURR'] =test_data['SK_ID_CURR']
df_pred.head()

,0,SK_ID_CURR
0,0.076265,100001
1,0.091212,100005
2,0.072289,100013
3,0.072499,100028
4,0.087044,100038


In [45]:
df_pred.set_index('SK_ID_CURR',inplace=True)
df_pred.rename(columns={0:'TARGET'},inplace=True)

In [46]:
df_pred.head()

,TARGET
SK_ID_CURR,
100001,0.076265
100005,0.091212
100013,0.072289
100028,0.072499
100038,0.087044


In [47]:
#df_pred.to_csv('test output/pred_one_lbm_first.csv',index=True)